# Association analysis brief overview

Association analysis is relatively straightforward concept. In addition, it is an unsupervised learning tool that looks for  patterns in data so there is relatively low requirement for initial data analysis and domain expertise. 

Definitions (ref Stanford CS246:Mining Massive Data Sets):

**Support** is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

**Confidence** is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

**Lift** is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

In [2]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from IPython.display import display, HTML
import sys
from itertools import combinations, groupby
from collections import Counter

# in this case I am not using hand-coded a prirori, but we will code it for 
# study purposes later in the notebook (stay tuned :) )
# update- the lib does not really work with 'large# (not really large) datasets
# had to cut it by hand
# from mlxtend.frequent_patterns import apriori
# from mlxtend.frequent_patterns import association_rules

In [3]:
# load the data
raw_df = pd.read_csv('export_with_basket_2017.csv', encoding = "utf-8")#, nrows = 100000)
raw_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID,SCM_N_ORDER_NR_1
0,341999621,80391417,0,06-JAN-17,1311010,62776372,12378259,1.00,61312,19997,...,139428.16 Royal Canin British Shorthair Adult ...,139428.16 Royal Canin British Shorthair Adult ...,Katzenfutter trocken,Royal Canin Rasse (Breed),British Shorthair,British Shorthair,Royal Canin,Royal Canin,3,80391417
1,348334117,81849313,0,02-FEB-17,2031175,111951408,2913095,1.00,70828,48693,...,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis","523432.4 Bozita Original - 12 kg + 1,8 kg gratis",Hundefutter trocken,Bozita,Bozita,Bozita,Doggy-Bozita,Dry Food Cat / Dog,2,81849313
2,348334114,81849313,0,02-FEB-17,1568093,53626549,2913095,1.00,56851,43068,...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,Hundesnacks,★ Lukullus,Lukullus feine Kauknochen,Lukullus feine Kauknochen,MATINA,snacks,2,81849313
3,354903151,83341895,0,02-MAR-17,1910589,38614848,7097883,0.25,51010,39043,...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3,83341895
4,354903151,83341895,0,02-MAR-17,1910589,38614828,7097883,0.25,51174,39043,...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3,83341895


In [4]:
# let us remove unsued columns:
col_list = ['ART_C_PROD_NUMBER', 'SCM_N_ORDER_NR_1']
orders = raw_df[col_list]

In [5]:
# those column labels are way to long
# Let us rename those
orders = orders.rename({"ART_C_PROD_NUMBER": "product_id",
                "SCM_N_ORDER_NR_1": "order_id"}, axis='columns')

In [6]:
# This does not work due to the bug in Pandas
# df.groupby(['CUS_N_KEY_CUSTOMER', 'ART_C_PROD_NUMBER']).sum().reset_index()
# df_pivot = df.pivot(index='CUS_N_KEY_CUSTOMER', columns='ART_C_PROD_NUMBER', values='SCM_D_QUANTITY')

## Data preparation

In [7]:
# Function that returns the size of an object in MB
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

In [8]:
print('orders -- dimensions: {0};   size: {1}'.format(orders.shape, size(orders)))
display(orders.head())

orders -- dimensions: (23768830, 2);   size: 380.30 MB


,product_id,order_id
0,139428,80391417
1,523432,81849313
2,468126,81849313
3,445245,83341895
4,445245,83341895


#### Convert order data into format expected by the association rules function

In [9]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
orders = orders.set_index('order_id')['product_id'].rename('item_id')
display(orders.head(10))
type(orders)

order_id
80391417    139428
81849313    523432
81849313    468126
83341895    445245
83341895    445245
89532203    451643
89532203    451643
89532203    199687
89532203    199737
86083656    183831
Name: item_id, dtype: int64

pandas.core.series.Series

### Display summary statistics for the order data

In [10]:
print('orders -- dimensions: {0};   size: {1};   unique_orders: {2};   unique_items: {3}'
      .format(orders.shape, size(orders), len(orders.index.unique()), len(orders.value_counts())))

orders -- dimensions: (23768830,);   size: 380.30 MB;   unique_orders: 5044124;   unique_items: 7514


## Association Rules Function

### Helper functions to the main association rules function

In [11]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]               

### Association rules function

In [12]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
   

    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

## Association Rules Mining

In [13]:
%%time
rules = association_rules(orders, 0.001)  # here it is important to get the right support, since we have relatively few items and large amount of users

Starting order_item:               23768830
Items with support >= 0.001:            6894
Remaining order_item:              23758584
Remaining orders with 2+ items:     4090755
Remaining order_item:              22806396
Item pairs:                         5943525
Item pairs with support >= 0.001:      81496

Wall time: 1min 31s


In [14]:
rules['item_A'].unique().size

4427

In [15]:
item_name = raw_df[['ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION']]
item_name.head()
#item_name = item_name.rename(columns={'ART_C_PROD_NUMBER':'item_id', 'ART_V_ART_DESCRIPTION':'item_name'})

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
0,139428,139428.16 Royal Canin British Shorthair Adult ...
1,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis"
2,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...
3,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...
4,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...


In [16]:
item_name = item_name.rename(columns={'ART_C_PROD_NUMBER':'item_id', 'ART_V_ART_DESCRIPTION':'item_name'})

In [17]:
item_name.head()

,item_id,item_name
0,139428,139428.16 Royal Canin British Shorthair Adult ...
1,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis"
2,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...
3,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...
4,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...


In [18]:
# not enough memory :O
# rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)

## Frequently bought together
> based on the mined association rules

In [19]:
rules_final = rules.sort_values(['lift'], ascending=False)

In [20]:
rules_final[rules_final['item_A'] != rules_final['item_B']].head()

,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
81466,657901,657900,54,0.001320,344,0.008409,379,0.009265,0.156977,0.142480,16.943362
22881,225411,49164,136,0.003325,1405,0.034346,252,0.006160,0.096797,0.539683,15.713232
34013,366454,366891,47,0.001149,485,0.011856,257,0.006282,0.096907,0.182879,15.425046
81465,657900,657901,46,0.001124,379,0.009265,344,0.008409,0.121372,0.133721,14.433234
81131,650625,650623,58,0.001418,503,0.012296,345,0.008434,0.115308,0.168116,13.672388


In [25]:
img_df = pd.read_csv('ps_id_to_img.csv', encoding = "utf-8")#, nrows = 100000)
name_to_url_lambda = lambda x: 'https://media.zooplus.com/bilder/' + x[x.find('.')-1] + '/140/'+ x

img_df['img_url'] = img_df['IMG_NAME'].apply(name_to_url_lambda)

In [43]:
thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=5>'+'Item A'+'</font>'))

for code in [225411]:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
    print(img_url)

display(HTML(images))

https://media.zooplus.com/bilder/5/140/22217_scherkoepfeschermaschinenoster_5.jpg


In [44]:
thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=5>'+'Item B'+'</font>'))

for code in [49164]:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
    print(img_url)

display(HTML(images))

https://media.zooplus.com/bilder/2/140/22216_PLA_Oster_Schermaschine_Golden_A5_2.jpg
